In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mping

def camera_calibration():
    for fname in images:
        img = mping.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        ret,corners = cv2.findChessboardCorners(gray,(8,6),None)
        
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            
            img = cv2.drawChessboardCorners(img,(8,6),corners,ret)
            
    return img,objpoints,imgpoints

def cal_undistort(img,objpoints,imgpoints):
    
    ret,mtx,dist,rvecs,tvecs = cv2.calibrateCamera(objpoints,imgpoints,gray.shape[::-1],None,None)
    undst = cv2.undistort(img,mtx,dist,None,mtx)
    
    return undist

def corners_unwarp(img,nx,ny,mtx,undist):
    gray = cv2.cvtColor(undist,cv2.COLOR_BGR2GRAY)
    
    ret,corners = cv2.findChessboardCorners(gray,(nx,ny),None)
    
    if ret == True:
        cv2.drawChessboardCorners(undist,(nx,ny),corners,ret)
        
        offset = 100
        
        img_size = (gray.shape[1],gray.shape[0])
        
        src = np.flaot32([corners[0],corners[nx - 1],corners[-1],corners[-nx]])
        
        dst = np.float32([[offset,offset],[img_size[0] - offset,offset],[img_size[0] - offset,img_size[1] - offset],[offset,img_size[1] - offset]])
        
        M = cv2.getPerspectiveTransform(src,dst)
        
        warped = cv2.warpPerspective(undist,M,img_size)
        
    return warped,M

def abd_sobel_thresh(img,orient = 'x',thresh_min = 0,thresh_max = 255):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray,cv2.CV64F,1,0))
        
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray,cv2.CV_64F,0,1))
        
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min)&(scaled_sobel<=thresh_max)] = 1
    
    return grad_output

def mag_thresh(img,sobel_kernel = 3,mag_thresh = (0,255)):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    
    return mag_output

def dir_threshold(img,sobel_kernel = 3, thresh = (0,np.pi/2)):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    
    absgraddir = np.arctan2(np.absolute(sobely),np.absolute(sobelx))
    
    binary_output = np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    
    return dir_output

def pipeline(img,s_thresh = (170,255),sx_thresh = (20,100)):
    img = np.copy(img)
    hls = cv2.cvtColor(img,cv2.COLOR_RGB2HLS).astype(np.float)
    
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    sobelx = cv2.Sobel(l_channel,cv2.CV_64F,1,0)
    abs_sobel = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobel/np.np.max(abs_sobelx))
    
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0])&(scaled_sobel <= sx_thresh[1])] = 1
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0])&(s_channel <= s_thresh[1])] = 1
    
    color_binary = np.dstack((np.zeros_like(sxbinary),sxbinary,s_binary))*255
    
    return color_binary

def window_mask(width,height,img_ref,center,level):
    
    output = np.zeros_like(img_ref)
    output[] = 1
    
    return output

def find_window_centroids(image,window_width,window_height,margin):
    
    window_centroids = []
    window = np.ones(window_width)
    
    l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)],axis = 0)
    l_center = np.argmax(np.convolve(window,l_sum)) - window_width/2
    r_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)],axis = 0)
    r_center = np.argmax(np.convolve(window,r_sum)) - window_width/2 + int(image.shape[1]/2)
    
    window_centroids.append((l_center,r_center))
    
    for level in range(1,(int)(image.shape[0]/window_height)):
        image_layer = np.sum(image[int(image.shape[0] - (level + 1)*window_height):int(image.shape[0] - level*window_height),:],axis = 0)
        cov_signal = np.convolve(window,image_layer)
        offset = window_width/2
        l_min_index = int(max(l_center + offset - margin,0))
        l_max_index = int(min(l_center + offset + margin,image.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index]) + l_min_index - offset
        
        r_min_index = int(max(r_center + offset - margin,0))
        r_max_index = int(min(r_center + offset + margin,image.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index]) + r_min_index - offset
        
    return window_centroids


    
    
    

ksize = 3

gradx = abs_sobel_thresh(image,orient = 'x',sobel_kernel = ksize, thresh = (0,255))
grady = abs_sobel_thresh(image,orient = 'y',sobel_kernel = ksize, thresh = (0,255))

mag_binary = mag_thresh(image,sobel_kernel = ksize,mag_thresh = (0,255))
dir_binary = dir_threshold(imgae, sobel_kernel = ksize, thresh = (0, np.pi/2))

combined = np.zeros_like(dir_binary)
combined[((gradx ==1) & (grady == 1))|((mag_binary == 1) & (dir_binary == 1))] = 1

window_centroids = find_window_centroids(warped,window_width,window_height,margin)

if len(window_centroids) > 0:
    
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)
    
    for level in range(0,len(window_centroids)):
        
        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        
        l_points[(l_points == 255)|((l_mask == 1))] = 255
        r_points[(r_points == 255)|((r_mask == 1))] = 255
        
    template = np.array(r_points + l_points,np,uint8)
    zero_channel = np.zeros_like(template)
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8)
    warpage = np.dstack((warped,warped,warped))*255
    output = cv2.addWeighted(warpage,1,template,0.5,0.0)
else:
    output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
    
ploty = np.linspace(0,719,num = 720)
quadratic_coeff  = 3e-4
    
leftx = np.array([200 + (y**2)*quadratic_coeff + np.random.randint(-50,high = 51) for y in ploty])
rightx = np.array([900 + (y**2)*quadratic_coeff + np.random.randint(-50,high = 51) for y in ploty])
    
leftx = leftx[::-1]
rightx = rightx[::-1]

mark_size = 3
plt.plot(leftx,ploty,'o',color = 'red',marksize = mark_size)
plt.plot(rightx,ploty,'o',color = 'blue',marksize = mark_size)
plt.xlim(0,1280)
plt.ylim(0,720)
plt.plot(left_fitx,ploty,color = 'green', linewidth = 3)
plt.plot(right_fitx,ploty,color = 'green', linewidth = 3)
plt.gca().invert_yaxis()

y_eval = np.max(ploty)
left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5)/np.absolute(2*left_fit[0])
right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5)/np.absolute(2*right_fit[0])


ym_per_pix = 30/720
xm_per_pix = 3.7/700

left_fit_cr = np.polyfit(ploty*ym_per_pix,leftx*xm_per_pix,2)
right_fit_cr = np.polyfit(ploty*ym_per_pix,rightx*xm_per_pix,2)

left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5)/np.absolute(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5)/np.absolute(2*right_fit_cr[0])   

print(left_curverad,'m',right_curverad,'m')

warp_zero = np.zeros_like(warped).astype(np.uint8)
color_warp = np.dstack((warp_zero,warp_zero,warp_zero))

pts_left = np.array([np.transpose(np.vstack([left_fitx,ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx,ploty])))])

cv2.fillPoly(color_warp,np.int_([pts]),(0,255,0))

newwarp = cv2.warpPerspective(color_warp,Minv,(image.shape[1],image.shape[0]))

result = cv2.addWeighted(undist,1,newarp,0.3,0)
plt.imshow(result)